<a href="https://colab.research.google.com/github/marcoandre1/sturdy-sniffle/blob/main/INF717_TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎬 Mise en contexte
Votre entreprise gère des centaines d'ordinateurs dans vos salles de serveurs.
L'entreprise veut faire de la maintenance préventive : elle veut inspecter l'état des cartes-mère (_motherboard_) de chaque ordinateur une fois par année.

Une partie de cette inspection est la détection de défauts au niveau de l'installation du _motherboard_. Bien qu'un néophyte n'est probablement pas apte à détecter les problèmes pour toutes sortes de _motherboard_, il est coûteux de faire l'inspection par un ou plusieurs experts.

**Vous cherchez une solution qui permettra à n'importe quel employé de facilement faire l'inspection sans expérience préalable.**

**Vous optez pour une solution via vision ordinateur et analyse d'image par IA**.

Le jeux de données _Motherboard Production Defects_ sera utilisée dans cette simulation.

<center>
<img src="https://i.ibb.co/DfPbKWbf/ezgif-4bf84575f81bc8.gif">\
</center>

# 👔 Travail à réaliser

À travers ce _notebook_, vous allez rencontrer **11 questions**.

Ces questions sont signalées d'un 💡. Lisez attentivement l'énoncé des questions avant de répondre.

Maintenant que vous êtes plus familier avec l'IA, plusieurs questions demanderont de faire une recherche. Ceci est pour évaluer votre compétence à acquérir de nouvelles connaissances en IA en entreprise. Vous avez le droit d'utiliser une IA telle que _ChatGPT_ pour démarrer vos recherches. **Cependant, vous devez écrire au moins une source qui ne provient pas d'une _IA_**. Par exemple, vous pouvez utiliser _ChatGPT_ pour trouver des pistes et demander des sources. Après, vous validez la source et la véracité des réponses de _ChatGPT_. Ceci devient une pratique commune en entreprise.

Les consignes pour la remise se trouvent sur _Moodle_.

**De plus**, lisez les commentaires dans les cellules de codes et étudiez bien la théorie et les méthodologies présentées dans ce _notebook_. Ce TP est également formatteur aux questions de compréhension théorique qui seront à l'examen final. Par exemple, vous allez explorer un modèle _YOLO_ à travers ce TP. Il est possible que l'examen contienne une question à propos de ce modèle.

# 0️⃣ : Basculer au _runtime_ T4 GPU.

Nous voulons profiter de l'accélération GPU pour l'entraînement. Changer le _runtime_ pour T4 GPU. Les bibliothèques que nous utiliseront plus tard détecterons la GPU et s'occuperont de lui délegué les calculs appropriés.

<center>
<img width=40% src="https://i.ibb.co/qLLwgMSZ/Screenshot-2025-06-17-040615.png">\
</center>

# 1️⃣ : Installation et importation des bibliothèques

In [ ]:
# Installation de la bibliothèque ultralytics sur la machine virtuelle.
# N'oubliez pas que la machine virtuelle est réinitialisée à chaque ouverture
# du notebook
!pip install ultralytics

# Importation de la bibliothèque dans l'instance courante de Python.
import ultralytics

# Vérification de l'initialisation de la bibliothèque suite à l'importation.
# Vous devriez voir le suivant :
# Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
# Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.5/112.6 GB disk)
ultralytics.checks()

# Lorsque vous voyez "CUDA", ça l'implique l'utilisation du GPU.
# CUDA est un API de Nvidia pour faire des calculs sur leurs GPUs.
# CUDA est très commun dans le milieu de l'IA.

Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/107.7 GB disk)


# 2️⃣ : Obtenir le jeu de données
Le _GitHub_ officiel du jeu de donnée est https://github.com/miraclefactory/motherboard-dataset?tab=readme-ov-file.

Cette page contient un lien vers le jeu de donnée. Le fichier est hébergé sur _Google Drive_.

Pour télécharger ce jeu dans notre environnement _Colab_, nous pouvons utiliser l'outil `gdown`. Celui-ci permet de facilement télécharger des fichiers de _Google Drive_ à travers un terminal.

Exécutez la cellule ci-dessous.

In [ ]:
!pip install -q gdown
!gdown "https://drive.google.com/uc?export=download&id=13-3N7eJnXEtKbyG6I2_GBXuYQqB_3PGt&confirm=t" -O DonnéesMotherboard_YOLO.zip
!unzip -q DonnéesMotherboard_YOLO.zip
!rm DonnéesMotherboard_YOLO.zip

Downloading...
From: https://drive.google.com/uc?export=download&id=13-3N7eJnXEtKbyG6I2_GBXuYQqB_3PGt&confirm=t
To: /content/DonnéesMotherboard_YOLO.zip
100% 232M/232M [00:05<00:00, 43.4MB/s]
replace Motherboard-13/README.roboflow.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Vous devriez maintenant voir le dossier _Motherboard-13_ dans votre répertoire temporaire _Colab_.

<center>
<img width=30% src="https://i.ibb.co/n8gPYHRt/Screenshot-2025-06-17-013342.png">
</center>

## 💡Question 1 : Terminal ou _Python_ ?
Notez l'utilisation du `!` dans la cellule ci-dessus. Ceci a un sens particulié dans une cellule de code _Colab_.

Expliquez ce le symbole `!` permet de faire dans _Colab_.
Expliquez aussi ce que fait `!pip install gdown`.

Écrivez votre réponse dans la cellule texte ci-dessous. Citez vos sources.

**Votre réponse ici**


# 2️⃣ : Connaître le jeu de données
La qualité et quantité des données est critique. Inspectons le jeu afin de souligner des points faibles qui pourront être pertinents pour expliquer la qualité de l'entraînement d'un modèle.

Le jeu de données que nous avons téléchargé est déjà structuré pour un modèle _YOLO11_.

_YOLO11_ sera apprivoisé dans une étape suivante.


## La structure _YOLO_

Un jeu de données d'images étiquettées n'est pas aussi simple que pour des données tabulaires. Dans le TP1, nous avions du texte avec une classe. Ici, nous avions une image associé à plusieurs détections de défaut. Chaque défaut a une classe et un emplacement dans l'image donné par une enveloppe de détection (_bounding box_).

Ultimement, nous voulons obtenir des _dataset PyTorch_ comme dans le TP1.
Il existe un module _Python_ pour des tâches utilisants un modèle _YOLO_. Ce module permet, entre autres, la création de _dataset_ à partir d'une arborescence de fichiers.

Au lieu de créer un ensemble d'entraînement, de validation et de tests avec plusieurs lignes de code, nous allons utiliser une fonction qui créer les trois. Cette fonction s'attend à une arborescence de fichiers structurés telle que nous avions téléchargée.

<center>
<img width=25% src="https://i.ibb.co/qF0M66Vk/arb.png">
</center>

Nous appelons ce genre de fonction un _loader_. Le _loader_ pour des modèles _YOLO_ s'occupe d'associer les fichiers d'étiquettes au images appropriées pour créer les _dataset_.

## Les annotations

Prenez un moment pour explorer la structure. Notez qu'il y a un répertoire pour les images et étiquettes (_labels_) pour chaque _split_ (entraînement, validation, test). Pour chaque image, il y a un fichier texte associé avec les annotations.

Par exemple, pour l'image `-----1---17_jpg.rf.5a20cfbb5df3a8e4fad656f2d15cdda6.jpg` du jeu de test, il y a les annotations suivantes :
<center>
<img width=25% src="https://i.ibb.co/PzgSNm3W/text.png">
</center>

Les modèles _YOLO_ sont fameux pour les tâches de **détection d'objets**. Dans l'exemple ci-dessus, nous voyons la structure typique pour étiquetter des images pour un modèle _YOLO_.

Voci la structure :
`<class_id> <x_centre> <y_centre> <largeur> <hauteur>`


* `<class_id>` : la classe de l'objet détecter (un entier)
* `<x_centre>`, `<y_centre>` : la position dans l'image où le centre de l'objet détecté se situe
* `<largeur>, <hauteur>` : dimension de la _bounding box_. Le centre de la _bounding box_ est à `(<x_centre>, <y_centre>)`.




## Les images

Observez les neuf premières images du jeu d'entraînement.

<center>
<img width=25% src="https://i.ibb.co/hJcB1bYy/images.png">
</center>

Vous pouvez ouvrir les images et les visualiser dans _Colab_.

Notez que les images ne contiennent pas les _bounding box_.

Ceci est sensé :

1.   Le modèle s'attend à des images de _motherboard_
2.   Le modèle génère ensuite les _bounding box_ et classe pour chaque objet détecté. Le modèle ne génère pas une nouvelle image.
3.   Les données génerées par le modèle peuvent _ensuite_ être utilisées pour créer une nouvelle image avec des _bounding box_ dessinés sur l'image originale.

Notez aussi que toutes les images sont de la même dimensions (largeur et hauteur). Après le passages des filtres du _CNN_, il faut avoir le même nombre de caractéristiques extraites. Il ne faut pas qu'une image en aille plus ou moins que les autres, car la tête _FC_ de classification a un nombre fixe de neurones intrants.

Finalement, notez qu'il semble avoir trois versions de la même image. Le plus évident étant la version décolorée. Il y a aussi une version avec le ton de couleur changé et des rotations.

Ceci nous mène au prochain point.



## Augmentation des données (_data augmentation_)

Le prétraitement (_preprocessing_) de nos données est une étape importante en IA. Nous voulons souvent retirer des points qui sont des abbérations statistiques, qui manque des valeurs pour certaines caractéristiques, etc. D'autres opérations comme la normalisation des données numériques est rarement ignoré.

En résumé, le prétraitement sert à mettre les données dans un état qui favorise l'apprentissage du modèle.

Pour des images, le pré-traitement inclut souvent d'uniformiser la dimension des images. D'autres opérations communes sont l'ajustement de l'orientation, l'équalisation des couleurs, etc.

Une autre opération commune est l'augmentation des données. Le but est de bonifier le nombre de données en créant des versions alternatives d'images dans un jeu de données initiales.

### Roboflow
Visitez le lien suivant :
https://universe.roboflow.com/yuelin-xin/motherboard-ptxx1/dataset/13/images

_Roboflow_ est un site populaire pour des jeux de données et modèles en analyse d'image. C'est semblable à _HuggingFace_, mais spécialisé pour les tâches de classification, détection d'objets, etc. pour les images.

Ici, vous pouvez visualiser les images des _splits_ avec ou sans _bounding box_ ajoutés (ils sont ajoutés procéduralement en lisant le fichier d'annotation !). Vous pouvez aussi voir le prétraitement et les augmentations efectuées pour obtenir chaque image.

<center>
<img width=60% src="https://i.ibb.co/h18Kv0qd/attributes.png">
</center>

Il est aussi possible d'utiliser _Roboflow_ pour créer un ensemble de données. Vous pouvez téléverser des images et chosir les prétraitement et augmentations que vous voulez appliquer. Vous pouvez aussi chosir le pourcentage des splits et _Roboflow_ va les générer avec une structure de fichiers pour YOLO ou autres modèles.

## 💡Question 2 : Pourquoi l'augmentation ?

Dans la cellule ci-dessous, écrivez :

**1)**

Sans faire une recherche, pourquoi pensez-vous que l'augmentation de données est une bonne pratique ?

Pensez à la pertinence de changer le ton de couleur et l'orientation, par exemple, et ce que ça peut impliquer sur la capacité du modèle à généraliser.

Vous ne serez pas pénalisé pour cette partie de la question si votre intuition est incorrecte.

\\
**2)**

Après avoir effectué une recherche, écrivez maintenant les réels avantages à l'augmentation des données. Citez vos sources.



**Vos réponses ici**

**1)**



**2)**

## Les classes

Visitez le lien suivant pour plus de statistiques à propos des données :

https://app.roboflow.com/testspace-so4mg/motherboard-ptxx1-leilu/health

_Roboflow_ génère ces statistiques pour nous.

## 💡Question 3 : À propos des classes...
Observez la distribution des classes. Voyez-vous le problème ?

Expliquez le problème et comment ce type de problème peut nuire à la performance d'un modèle entraîné.

Répondez dans la cellule texte ci-dessous. Citez vos sources.

**Votre réponse ici**

##💡Question 4 : Métriques de classification en IA

Il y a quatres métriques pour évaluer la performance d'un modèle de classification.

| Anglais       | Français                                |
| ------------- | --------------------------------------- |
| **Accuracy**  | **Exactitude** ou **Précision globale** |
| **Precision** | **Précision**                           |
| **Recall**    | **Rappel**                              |
| **F1-score**  | **Score F1**                            |

**1)**

Dans la cellule ci-dessous, décrivez ce que chaque métrique représente. Citez vos sources.

\\
**2)**

Expliquez pourquoi le score F1 est un meilleur métrique lorsqu'il y a un déséquilibre des classes. Citez vos sources.

**Vos réponses ici**

**1)**



**2)**

#3️⃣ : Sélection du modèle _YOLO_ préentraîné

Vous pouvez en apprendre plus sur _YOLO11_ en visitant la page suivante : https://docs.ultralytics.com/models/yolo11/#models

Notez qu'il existe des modèles pour plusieurs tâches (détection, segmentation, classification globale, etc.). La tâche par défaut est la détection.

Il existe cinq taille/compléxité de modèles _YOLO11_ pour la détection : nano, small, medium, large, x-large.

| Model       | Paramètres (Millions) |
| ----------- | --------------------- |
| **YOLO11n** | 2.6 M                 |
| **YOLO11s** | 9.4 M                 |
| **YOLO11m** | 20.1 M                |
| **YOLO11l** | 25.3 M                |
| **YOLO11x** | 56.9 M                |

Pour commencer, vous allez utiliser la version `YOLO11n` (nano). Plus bas, vous allez échanger le modèle pour la version x-large et comparer les résultats et le temps d'entraînement.


##💡Question 5 : Code d'instanciation du modèle

Complétez la cellule de code ci-dessous. Vous devez trouver la bonne `string` pour charger un _YOLO11n_.

Vous devriez voir un nouveau fichier `.pt` dans votre répertoire temporaire _Colab_.

In [ ]:
from ultralytics import YOLO

model_nano = YOLO("complétez ici") # s'occupe aussi de télécharger les paramètres du modèle venant du préentraînement
print(model_nano.info())

YOLO11x summary: 357 layers, 56,966,176 parameters, 0 gradients, 196.0 GFLOPs
(357, 56966176, 0, 195.9587328)


##💡Question 6 : Tester le modèle préentraîné

Exécutez la cellule de code suivante pour tester votre _YOLO11n_.

Notez les statistiques de la sortie. Il y a le nombre d'occurence par classes, le temps d'inférence, etc.

In [ ]:
!wget https://i.ibb.co/8nCZJByL/erepublic-brightspotcdn.jpg -O image.jpg # ligne pour télécharger l'image de test

results = model_nano("image.jpg")
results[0].show()

Dans la prochaine cellule de code, créez un nouveau modèle _YOLO11_ x-large. Faites le même test et observez la différence.

In [ ]:
# Votre réponse ici

Finalement, dans la cellule texte ci-dessous, écrivez le temps d'inférence obtenu entre le modèle nano et le modèle x-large. De plus, écrivez à propos de la différence de qualité entre deux résultats.

**Vos réponses ici**

##💡Question 7 : Test sur un _motherboard_

Dans la cellule de code ci-dessous, écrivez le code nécessaire pour tester le `model_xlarge` et afficher les résultats avec une image de _motherboard_ comme intrant.

Utilisez une image dans le répertoire _Motherboard-13_. Prenez la première image de la split _train_.

In [ ]:
# Votre réponse ici

Sans faire une recherche, selon vous, pourquoi est-ce que le résultat avec un _motherboard_ n'est pas bon ?

Expliquez dans la cellule texte suivante.

**Votre réponse ici**

##💡Question 8 : _COCO_ (_Common Objects in Context_)
Le jeu de données _COCO_ a été utilisé pour le préentraînement de tous les modèles _YOLO_.

**1)**

Dans la cellule suivante, après une recherche, nommez dix (10) classes d'objets trouvées dans _COCO_. Citez vos sources.

\\
**2)**

Après avoir consultez la liste des classes, pourquoi croyez-vous maintenant que le modèle ne fonctionne pas pour la détection sur un _motherboard_ ?

Qu'est-ce qui doit être fait pour adapter le modèle pour notre tâche de détections de défauts ?

\\
**3)**

Quelle est la pertinence de commencer à partir d'un modèle préentraîné comme `yolo11n` ?

Expliquez l'importance des filtres de convolution préoptimisés. Qu'est-ce que ces filtres ont appris à faire pendant le préentrainement sur les données de _COCO_ ?

\\
**4)**

Dans le TP1, nous avons garder l'encodeur du _transformer BARTHez_ préentraîné pour faire la classification de tickets de support.

Ici, entre la section _CNN_ (souvent nommé _backbone_) et la tête de classification, lequel voulons nous garder pour le _fine-tuning_ de notre tâche de détection de défauts ?

Citez vos sourcre.

\\
**5)**

Si nous voulons détecter des chats et des chiens, au lieu de défauts _motherboard_, est-il nécessaire de faire un _fine-tuning_ du modèle ?

**Vos réponses ici**

**1)**



**2)**



**3)**



**4)**



**5)**

# 4️⃣ : _Fine-tuning_ des modèles

Le module `ultralytics` offre un API très simple pour l'entraînement et la validation de modèle _YOLO_. En une seule fonction, vous fournir les données, configurer les hyperparamètres et entraîner. Les _dataset Pytorch_ seront créer pour vous et utilisés sous le capot.

Les objets _YOLO_ (comme `model_nano`) ont accès à la méthode `train(...)`.

Visitez la page de documentation officiel pour en apprendre plus sur comment charger les données et choisir les hyperparamètres : https://docs.ultralytics.com/modes/train/#train-settings

Les paramètres de la fonction `train(...)` importants pour ce TPs sont :
*   data   (le fichier `.yaml` se trouve dans le dossier _Motherboard-13_)
*   name   (vous choisissez)
*   epochs (utilisez une valeur de 5)
*   imgsz  (utilisez une valeur de 128)
*   batch  (utilisez une valeur de 16)
*   seed   (par defaut à 0. Utilisez cette valeur par défaut)
*   pretrained (par défaut à `True`. C'est ce que nous voulons.)

À la fin de l'entraînement, nous obtenons un dossier avec les nouveaux poids (paramètres) du modèle. Aussi, des graphiques pour le score F1, et autres métriques. Vous obtenez d'autres fichiers avec d'autres statistiques pertinentes. Explorez ces fichiers à votre guise.

<center>
<img width=20% src="https://i.ibb.co/ZzJZF9FC/Screenshot-2025-06-17-170933.png">
</center>

##💡Question 9 : Code d'entraînement

Entraînez le modèle `yolo11n` et le modèle `yolo11x`. Utilisez les valeurs de paramètres données plus haut.

Attendez-vous que le processus soit **très long**.

Pour la version nano, vous devriez obtenir le graphique `results.png` suivant :

<center>
<img width=75% src="https://i.ibb.co/LzPmQgkd/Screenshot-2025-06-17-171622.png">
</center>

Pour x-large :

<center>
<img width=75% src="https://i.ibb.co/JWkGPBTt/Screenshot-2025-06-17-180632.png">
</center>

Notez le métrique _mAP50-95_. Ceci est un métrique utilisé en détection d'objets. Vous pouvez en lire davantage, mais une valeur plus haut de 0.5 (50%) est souvent considérée très bon.

In [ ]:
# yolo11n

In [ ]:
# yolo11x

##💡Question 10 : Test

Testez votre modèle nano et x-large encore une fois sur la première image de _motherboard_ du _split train_.



##💡Question 11 : Comparaison nano et x-large.

Quel est le compromis (_tradeoff_) entre un petit modèle et un plus large ?

Pensez à la taille mémoire du modèle, la vitesse d'entraînement et d'inférence, les métriques de performances.

Essayez une ou deux époque de `yolo11n` et de `yolo11x` et appuyez votre réponse avec les résultats de vos expériences.

Répondez dans la cellule ci-dessous.

In [ ]:
# Votre réponse ici

## 💡Question 12 : YOLO11 (CNN) contre DETR (ViT)

Pour compléter votre apprentissage en méthodes IA pour le traitement d'image, vous devez faire une recherche et comparer la méthode _CNN_ avec la méthode _Vision Transformer_.

Les deux méthodes coexistents dans l'industrie ; ils attaquent des besoins différents.

Dans une cellule texte ci-dessous, répondez aux questions inscrites.

Suite à cette recherche, vous devriez être en mesure de recommander le plus approprié selon la nature du problème. N'oubliez pas de citez vos sources.


**1)**

En quelle année est-ce que YOLO11 est devenu disponible au public ?

En quelle année est-ce que DETR est devenu disponible au public ?


**2)**

Quelle organisation/entreprise a développé YOLO11 ?

Quelle organisation/entreprise a développé DETR ?


**3)**

Indiquez un avantage que YOLO11 (grâce à son coeur _CNN_) a en comparant avec DETR.

Indiquez un avantage que DETR (grâce à son coeur _ViT_) a en comparant avec YOLOv8.


**4)**

Donnez une situation ou vous recommanderiez un _CNN_ pour la classification d'images au lieu d'un _ViT_.

Donnez une situation ou vous recommanderiez un _ViT_ pour la classification d'images au lieu d'un _CNN_.


# 💡 Bonus : Déploiement

Utilisez la bibliothèque `gradio` pour deployer un service web avec interface pour utilisez votre modèle.

Vous devriez obtenir un URL public que vous pouvez partager.

Par exemple, https://fbeada3b7870419550.gradio.live/

In [ ]:
# Votre réponse ici